## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as k
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.applications import Xception

input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model=Xception(include_top=False, weights='imagenet', input_tensor=input_tensor,pooling=None, classes=10)


'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()


Using TensorFlow backend.


83689472/83683744 [==============================] - 154s 2us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
________________________

block8_sepconv3 (SeparableConv2 (None, 2, 2, 728)    536536      block8_sepconv3_act[0][0]        
__________________________________________________________________________________________________
block8_sepconv3_bn (BatchNormal (None, 2, 2, 728)    2912        block8_sepconv3[0][0]            
__________________________________________________________________________________________________
add_7 (Add)                     (None, 2, 2, 728)    0           block8_sepconv3_bn[0][0]         
                                                                 add_6[0][0]                      
__________________________________________________________________________________________________
block9_sepconv1_act (Activation (None, 2, 2, 728)    0           add_7[0][0]                      
__________________________________________________________________________________________________
block9_sepconv1 (SeparableConv2 (None, 2, 2, 728)    536536      block9_sepconv1_act[0][0]        
__________

## 添加層數

In [2]:
x = model.output

'''可以參考Cifar10實作章節'''
x=GlobalAveragePooling2D()(x)
x=Dense(128,kernel_initializer ='uniform', activation='relu')(x)
x=Dropout(0.1)(x)


predictions = Dense(10,kernel_initializer ='uniform', activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 136


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [7]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 192s 4ms/step - loss: 1.2381 - accuracy: 0.5702
Epoch 2/100
50000/50000 [==============================] - 173s 3ms/step - loss: 0.7412 - accuracy: 0.7596
Epoch 3/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.5688 - accuracy: 0.8159
Epoch 4/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.4588 - accuracy: 0.8507
Epoch 5/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.3726 - accuracy: 0.8788
Epoch 6/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.3080 - accuracy: 0.8995
Epoch 7/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.2566 - accuracy: 0.9158
Epoch 8/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.2106 - accuracy: 0.9311
Epoch 9/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.1865 - accuracy: 0.9402
Epoch 10/100
50000/50000 [===========

50000/50000 [==============================] - 170s 3ms/step - loss: 0.0232 - accuracy: 0.9926
Epoch 77/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.0276 - accuracy: 0.9920
Epoch 78/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.0204 - accuracy: 0.9938
Epoch 79/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.0279 - accuracy: 0.9917
Epoch 80/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.0258 - accuracy: 0.9928
Epoch 81/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.0293 - accuracy: 0.9919
Epoch 82/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.0257 - accuracy: 0.9940
Epoch 83/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.0231 - accuracy: 0.9935
Epoch 84/100
50000/50000 [==============================] - 171s 3ms/step - loss: 0.0222 - accuracy: 0.9935
Epoch 85/100
50000/50000 [===============